In [3]:
import numpy as np

In [10]:
results_file = './saved_results/PJR_replay_1.0/PJR_acc_rocauc_tr_time_best_epoch_1.npz'
results = np.load(results_file, allow_pickle=True)

In [15]:
accuracy = results['accuracy']
rocauc = results['rocauc']
tr_time = results['tr_time']
best_epochs = results['best_epochs']


print(f'Accuracy {accuracy} \n\nand ROCAUC {rocauc}')

Accuracy {'2018-01': [0.9218346253229974], '2018-02': [0.9394278821299328], '2018-03': [0.9432070562852973], '2018-04': [0.9573159099580313], '2018-05': [0.9580985915492958], '2018-06': [0.9597710601624947], '2018-07': [0.9623283817240603], '2018-08': [0.961882904682316], '2018-09': [0.9625140194978863], '2018-10': [0.9640039268443442], '2018-11': [0.9635879764944775], '2018-12': [0.9685891795323041]} 

and ROCAUC {'2018-01': [0.9794730392156863], '2018-02': [0.9858464120175797], '2018-03': [0.986499843364934], '2018-04': [0.9908984370204862], '2018-05': [0.9916232373495568], '2018-06': [0.9920307392897942], '2018-07': [0.9932663406467159], '2018-08': [0.9931048584962039], '2018-09': [0.9936209372738639], '2018-10': [0.9939147755184543], '2018-11': [0.9946698110472352], '2018-12': [0.9955782059196869]}


In [4]:
def get_month_data(data_dir, month, train=True):
    
    if train:
        data_dir = data_dir + str(month) + '/'
        XY_train = np.load(data_dir + 'XY_train.npz')
        X_tr, Y_tr = XY_train['X_train'], XY_train['Y_train']

        return X_tr, Y_tr
    else:
        data_dir = data_dir + str(month) + '/'
        XY_test = np.load(data_dir + 'XY_test.npz')
        X_test, Y_test = XY_test['X_test'], XY_test['Y_test']



def get_task_partial_joint_training_data(data_dir, task_months, replay_portion, mlp_net=False):
    
    #X_tr, Y_tr, X_val, Y_val = get_month_data(data_dir, task_months[-1])
    X_tr, Y_tr = get_month_data(data_dir, task_months[-1])
    print(f'Current Task month {task_months[-1]} data X {X_tr.shape} Y {Y_tr.shape}')
    for month in task_months[:-1]:
        #pre_X_tr, pre_Y_tr, pre_X_val, pre_Y_val = get_month_data(data_dir, month)
        pre_X_tr, pre_Y_tr = get_month_data(data_dir, month)
        pre_X_tr, pre_Y_tr = get_partial_data(pre_X_tr, pre_Y_tr, replay_portion)
        
        print(f'previous month {month} data X {pre_X_tr.shape} Y {pre_Y_tr.shape}')
        
        X_tr, Y_tr = np.concatenate((X_tr, pre_X_tr)), np.concatenate((Y_tr, pre_Y_tr))
        #X_val, Y_val = np.concatenate((X_val, pre_X_val)), np.concatenate((Y_val, pre_Y_val))
    
    if not mlp_net:
        X_train, Y_train  = reshape_features(X_tr, Y_tr)
        X_valid, Y_valid = reshape_features(X_val, Y_val)
    else:
        X_train, Y_train  = X_tr, Y_tr
        #X_valid, Y_valid = X_val, Y_val
    print(f'X_train {X_train.shape} Y_train {Y_train.shape}\n')
    #print(f'X_valid {X_valid.shape} Y_valid {Y_valid.shape}\n')
    
    #return X_train, Y_train, X_valid, Y_valid
    return X_train, Y_train



#pre_X_tr, pre_Y_tr = get_month_data(data_dir, month)

#X_train, Y_train = get_task_partial_joint_training_data(data_dir, task_months, replay_portion, mlp_net=True)

In [5]:
data_dir = '../../../ember2018/month_based_processing/'

all_task_months = ['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                   '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12']

month = all_task_months[0]
pre_X_tr, pre_Y_tr = get_month_data(data_dir, month)


In [9]:
pre_Y_tr

array([1., 1., 0., ..., 0., 1., 0.], dtype=float32)

In [16]:
26495 - np.ceil(29423 * 0.9)

14.0

In [12]:
len(np.where(pre_Y_tr == 0)[0]), len(np.where(pre_Y_tr == 1)[0])

(26495, 29227)